# Синтаксис. 
# Дерево зависимостей (Dependency parsing).
Зависимостный парсинг - одна из немногих задач, где лингвистика и компьютерные науки продуктивно взаимодействуют. Основную теорию тут разрабатывают лингвисты, они же размечают корпусы, а модели построены на самых последних разработках в области машинного обучения.

Что мы уже научились делать:
1) Разбить текст на предложения
2) Разбить предложение на токены/слова
3) Сделать морфологический анализ токенов
____
Следующий этап - это определение связей между токенами, то есть синтаксис. Для этого нам и потребуются синтаксические парсеры.  
Существуют разные варианты парсеров и сегодня мы познакомимся с самыми известными из них. А так же рассмотрим основные разницы между ними и какой функционал у них есть.



## Принцип работы парсера

**На вход:** текст (последовательность токенов).   
**На выход:** ориентированный граф, в котором узлами являются наши токены, а ребрами являются разные виды связей между токенами.  
  
Порядок действий парсера, за каждым из этих этапов стоит не самая простая модель:
1) Разбивает текст на предложения
2) Разбивает предложение на токены/слова
3) Производит морфологический анализ токенов
4) Анализ зависимостей между токенами, синтаксический анализ.

Раньше пункт 4 выполнялся с помощью множества правил, разработанных лингвистами. Но данный подход оказался не оптимальным, так как языки меняются со временем, появляются новые слова и конструкции. А даже небольшие изменения, как правило приводят к огромной работе по изменению такого парсера.

Поэтому перешли к модельному подходу. Теперь лингвисты размечают корпуса текстов, где указывают морфологические признаки и зависимости в предложениях.
На этих данных обучаются сложные модели, задача которых, научиться искать эти связи во входящих данных. Как правило, эти модели называются декодерами.
На вход такая модель получает последовательность токенов с их признаками, а на выходе она дает нам граф.


Вот так примерно выглядит результат работы парсера:  
![](https://habrastorage.org/getpro/habr/post_images/686/c78/066/686c780661b296250d53cba054317a18.png)

## Категории парсеров
Большинство парсеров можно поделить на две категори:  
- **Transition-based decoders**  
    Токены обрабатываются нейронкой по порядку слева на право. Такой подход является более быстрым, но меннее точным. Падение точности ощутимо на длинных зависимостях, когда между токенами большое расстояние.
- **Graph-based decoders**
    Cтроится граф, узлами которого являеются наши токены. Все токены соеденены друг с другом ребрами. Затем модель определяет какие ребра можно убрать, а у оставшихся определяет тип зависимости и направленность. Такой подход является более точным, но требует больших вычислений.

## Примеры парсеров
На этом семинаре мы рассмотрим следующие парсеры:
1) SpaCy
2) UDPipe
3) Stanza

# Import libs

In [298]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# [Spacy](https://spacy.io/)
Библиотека для продвинутого NLP. Очень много полезного функционала прям из коробки.  
Так как предобученные модели могут занимать много места, а вы ими даже не будете пользоваться, то по умолчанию после установки spacy на вашем компьютере нет никаких моделей ни для какого языка.  
[Но есть инструкция, как скачать и установить модель для необходимого вам языка.](https://spacy.io/models)


In [2]:
!pip install spacy

In [299]:
import spacy
from spacy import displacy
from spacy.tokens import Span

In [6]:
# Скачиваем модель для английского языка
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 2.4 MB/s eta 0:00:01
You should consider upgrading via the '/Users/u14510182/Documents/python_for_nlp_stud/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
# Скачиваем модель для русского языка
!python -m spacy download ru_core_news_sm

     |████████████████████████████████| 15.3 MB 2.3 MB/s eta 0:00:01
You should consider upgrading via the '/Users/u14510182/Documents/python_for_nlp_stud/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


## Родной парсер и функционал
Лемматизация, токенизация, морфологический анализ, частеречная разметка + **Синтаксис (Дерево зависимостей)**  
[Подробная инструкция про лингвистические возможности.](https://spacy.io/usage/linguistic-features#pos-tagging)

In [300]:
eng_default = spacy.load("en_core_web_sm")

In [301]:
eng_default.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [302]:
doc = eng_default("Apple is looking at buying U.K. startup for $1 billion")

In [303]:
type(doc), doc

(spacy.tokens.doc.Doc, Apple is looking at buying U.K. startup for $1 billion)

In [304]:
type(doc[2]), doc[2]

(spacy.tokens.token.Token, looking)

In [305]:
type(doc[2].morph), doc[2].morph

(spacy.tokens.morphanalysis.MorphAnalysis,
 Aspect=Prog|Tense=Pres|VerbForm=Part)

In [306]:
def get_features(token):
    features = {
        'text': token.text,
        'lemma': token.lemma_,
        'pos': token.pos_,
        'tag': token.tag_,
        'morph': token.morph,
        'dep': token.dep_,
        'head': token.head,
        'shape': token.shape_,
        'is_alpha': token.is_alpha,
        'is_stop': token.is_stop
    }
    return features

eng_doc_df = pd.DataFrame([get_features(token) for token in doc])
eng_doc_df

,text,lemma,pos,tag,morph,dep,head,shape,is_alpha,is_stop
0,Apple,Apple,PROPN,NNP,(Number=Sing),nsubj,looking,Xxxxx,True,False
1,is,be,AUX,VBZ,"(Mood=Ind, Number=Sing, Person=3, Tense=Pres, ...",aux,looking,xx,True,True
2,looking,look,VERB,VBG,"(Aspect=Prog, Tense=Pres, VerbForm=Part)",ROOT,looking,xxxx,True,False
3,at,at,ADP,IN,(),prep,looking,xx,True,True
4,buying,buy,VERB,VBG,"(Aspect=Prog, Tense=Pres, VerbForm=Part)",pcomp,at,xxxx,True,False
5,U.K.,U.K.,PROPN,NNP,(Number=Sing),dobj,buying,X.X.,False,False
6,startup,startup,NOUN,NN,(Number=Sing),dobj,buying,xxxx,True,False
7,for,for,ADP,IN,(),prep,startup,xxx,True,True
8,$,$,SYM,$,(),quantmod,billion,$,False,False
9,1,1,NUM,CD,(NumType=Card),compound,billion,d,False,False


In [307]:
spacy.explain('nsubj')

'nominal subject'

In [308]:
spacy.explain('NNP')

'noun, proper singular'

**Навигация по дереву: вершины и зависимые.**  
Используется терминология "head" (вершина) и "child" (зависимое)

In [309]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Apple nsubj looking VERB []
is aux looking VERB []
looking ROOT looking VERB [Apple, is, at]
at prep looking VERB [buying]
buying pcomp at ADP [U.K., startup]
U.K. dobj buying VERB []
startup dobj buying VERB [for]
for prep startup NOUN [billion]
$ quantmod billion NUM []
1 compound billion NUM []
billion pobj for ADP [$, 1]


In [165]:
print(f'{doc[2] = }')
print('Left childs:', list(doc[2].lefts))
print('Right childs:', list(doc[2].rights))
print('Count left childs:', doc[2].n_lefts)  
print('Count right childs:', doc[2].n_rights) 

doc[2] = looking
Left childs: [Apple, is]
Right childs: [at]
Count left childs: 2
Count right childs: 1


Визуализируем дерево:

In [310]:
doc = eng_default("Autonomous cars shift insurance liability toward manufacturers")
displacy.render(doc, style='dep')

## NER - Named Entity Recognition

In [312]:
doc = eng_default("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [313]:
displacy.render(doc, style="ent")

In [ ]:
eng_default.pe

In [314]:
eng_default.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [316]:
spacy.explain('LAW')

'Named documents made into laws.'

In [317]:
def get_token_entities(token):
    features = {
        'text': token.text,
        'ent_iob': token.ent_iob_,
        'ent_type': token.ent_type_
    }
    
    if features['ent_iob'] == 'I' :
        features['iob_description'] =  'Inside an entity'
    elif features['ent_iob'] == 'O' :
        features['iob_description'] =  'Outside an entity'
    else:
        features['iob_description'] =  'Beginning of an entity'
        
    features['type_description'] = None if features['ent_iob'] == 'O' else spacy.explain(features['ent_type'])
    return features

eng_doc_df = pd.DataFrame([get_token_entities(token) for token in doc])
eng_doc_df

,text,ent_iob,ent_type,iob_description,type_description
0,Apple,B,ORG,Beginning of an entity,"Companies, agencies, institutions, etc."
1,is,O,,Outside an entity,None
2,looking,O,,Outside an entity,None
3,at,O,,Outside an entity,None
4,buying,O,,Outside an entity,None
5,U.K.,B,GPE,Beginning of an entity,"Countries, cities, states"
6,startup,O,,Outside an entity,None
7,for,O,,Outside an entity,None
8,$,B,MONEY,Beginning of an entity,"Monetary values, including unit"
9,1,I,MONEY,Inside an entity,"Monetary values, including unit"


In [318]:
doc = eng_default("Twitter permanently suspends President Donald Trump")
displacy.render(doc, style="ent")

In [319]:
twitter_ent = Span(doc, 0, 1, label="ORG")
doc.ents = list(doc.ents) + [twitter_ent]
displacy.render(doc, style="ent")

In [320]:
president_ent = Span(doc, 3, 4, label="POLITIC")
doc.ents = list(doc.ents) + [president_ent]
displacy.render(doc, style="ent")

## Практика
1) Дадим на разбор предложение с синтаксической неоднозначностью. Какой анализ предлагает SpaCy?
    + John saw the man on the mountain with a telescope.
    + I'm glad I'm a man, and so is Lola.
    + больше примеров: https://en.wikipedia.org/wiki/Syntactic_ambiguity
2) Проверьте, справляется ли SpaCy с эллипсисом:
    + John can play the guitar; Mary can, too.
3) Протестируйте SpaCy для русского языка. Сравните какие типы зависимостей есть в русском и английском языке, отличаются ли они.

# UDPipe
SpaCy интегрирует в свою модуль не только собственные разработки, но и достижения сторонних разработчиков и организаций. Так они добавили свою обертку над синтаксическим парсером UDPipe, которая намного удобней своих аналогов.  

[**Universal Dependencies**](https://universaldependencies.org/) — это проект по унификации разметки синтаксических корпусов (трибанков) в рамках грамматики зависимостей. В русском языке количество типов синтаксических связей ограничено — подлежащее, сказуемое и т.д. В английском то же самое, но набор уже другой. Например, там появляется артикль, который тоже надо как-то маркировать. Если бы мы хотели написать волшебный парсер, который мог бы обрабатывать все языки, то довольно быстро уперлись бы в проблемы сопоставления разных грамматик. Героическим создателям Universal Dependencies удалось договориться между собой и разметить все корпусы, которые имелись в их распоряжении, в едином формате.

Для того, чтобы разметить что-то новое, нужно обучить модель на размеченном корпусе. Таких корпусов для русского языка есть несколько (можно посмотреть на сайте Universal Dependencies). Основные и самые большие размеченные корпуса:
+ Syntagrus
+ Taiga

[Очень хороший туториал по UDPipe.](https://habr.com/ru/company/sberbank/blog/418701/)

[**Список всех отношения в Universal Dependencies для русского языка.**](https://universaldependencies.org/ru/index.html)

## UDPipe Online
Перейдем по ссылке: http://lindat.mff.cuni.cz/services/udpipe/run.php

+ Сначала надо выбрать корпус syntagrus. 
+ Потом можно вводить свои предложения и рисовать для них таблицы и деревья.

## Spacy UDPipe

In [76]:
!pip install spacy_udpipe

You should consider upgrading via the '/Users/u14510182/Documents/python_for_nlp_stud/venv/bin/python -m pip install --upgrade pip' command.


In [78]:
import spacy_udpipe

In [89]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
spacy_udpipe.download("ru") # Скачиваем модель UDPipe для русского языка. Скачиваем только одни раз!

Downloaded pre-trained UDPipe model for 'ru' language


In [322]:
ru_udpipe = spacy_udpipe.load("ru")
ru_udpipe.pipe_names

[]

In [329]:
txt = 'Я был в деревне целый день. И завтра тоже там буду.'
txt = 'Миновав галерею, я приблизился к арочному проему, за которым открывался зал с высоким сводом, и остановился.'
doc = ru_udpipe(txt)

In [324]:
type(doc), doc

(spacy.tokens.doc.Doc, Я был в деревне целый день. И завтра тоже там буду.)

In [325]:
type(doc[1]), doc[1]

(spacy.tokens.token.Token, был)

In [330]:
sents = list(doc.sents)
sents

[Миновав галерею, я приблизился к арочному проему, за которым открывался зал с высоким сводом, и остановился.]

In [331]:
displacy.render(sents[0], style='dep')

In [332]:
doc_df = pd.DataFrame([get_features(token) for token in sents[0]])
doc_df

,text,lemma,pos,tag,morph,dep,head,shape,is_alpha,is_stop
0,Миновав,миновать,VERB,,"(Aspect=Imp, Tense=Past, VerbForm=Conv, Voice=...",advcl,приблизился,Xxxxx,True,False
1,галерею,галерея,NOUN,,"(Animacy=Inan, Case=Acc, Gender=Fem, Number=Sing)",obj,Миновав,xxxx,True,False
2,",",",",PUNCT,,(),punct,Миновав,",",False,False
3,я,я,PRON,,"(Case=Nom, Number=Sing, Person=1)",nsubj,приблизился,x,True,True
4,приблизился,приблизиться,VERB,,"(Aspect=Perf, Gender=Masc, Mood=Ind, Number=Si...",ROOT,приблизился,xxxx,True,False
5,к,к,ADP,,(),case,арочному,x,True,True
6,арочному,арочный,NOUN,,"(Animacy=Anim, Case=Dat, Gender=Masc, Number=S...",obl,приблизился,xxxx,True,False
7,проему,проема,DET,,"(Case=Dat, Gender=Masc, Number=Sing)",det,приблизился,xxxx,True,False
8,",",",",PUNCT,,(),punct,открывался,",",False,False
9,за,за,ADP,,(),case,которым,xx,True,True


In [333]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Миновав advcl приблизился VERB [галерею, ,]
галерею obj Миновав VERB []
, punct Миновав VERB []
я nsubj приблизился VERB []
приблизился ROOT приблизился VERB [Миновав, я, арочному, проему, остановился, .]
к case арочному NOUN []
арочному obl приблизился VERB [к]
проему det приблизился VERB [открывался]
, punct открывался VERB []
за case которым PRON []
которым obl открывался VERB [за]
открывался acl:relcl проему DET [,, которым, зал]
зал nsubj открывался VERB [сводом]
с case сводом NOUN []
высоким amod сводом NOUN []
сводом nmod зал NOUN [с, высоким]
, punct остановился VERB []
и cc остановился VERB []
остановился conj приблизился VERB [,, и]
. punct приблизился VERB []


# [Stanza](https://stanfordnlp.github.io/stanza/index.html)
Данная библиотека разработана **Stanford NLP Group**. В основе ее лежат нейросетевые модели (Pytorch). Так же библотека является оберткой над их известным java фреймворком для работы с текстом - **CoreNLP**. `Stanza` так же придерживалась стандарта разметки Universal Dependencies.  

Что входит в пайплайн `stanza`:
- Токенизация  
- Лемматизация (token.lemma and token.lemma_)
- Морфологический разбор = Part-of-speech tagging (token.tag, token.tag_, token.pos, token.pos_)
- Синтаксический анализ = Dependency parsing (token.dep, token.dep_, token.head)
- Распознавание сущностей = Named entity recognition (doc.ents, token.ent_type, token.ent_type_, token.ent_iob, token.ent_iob_)
- Разбиение на предложения = Sentence segmentation (doc.sents)

Так как под капотом здесь Pytorch, вы можете получить большой выигрыш в скорости обработки, если задействуете GPU.

In [197]:
!pip install stanza

You should consider upgrading via the '/Users/u14510182/Documents/python_for_nlp_stud/venv/bin/python -m pip install --upgrade pip' command.


In [198]:
import stanza

/Users/u14510182/Documents/python_for_nlp_stud/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [199]:
stanza.download("ru") # Загружаем один раз.

2022-10-17 03:38:10 INFO: Downloading default packages for language: ru (Russian) ...
2022-10-17 03:38:27 INFO: Finished downloading models and saved to /Users/u14510182/stanza_resources.


## Spacy Stanza

In [193]:
!pip install spacy-stanza

     |████████████████████████████████| 691 kB 770 kB/s eta 0:00:01
     |████████████████████████████████| 133.8 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 484 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 216 kB 3.3 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212391 sha256=13f6bb901dd0798b46f0afc7f64f36e22801f9a097313a2fa18fc657e4ffd849
  Stored in directory: /Users/u14510182/Library/Caches/pip/wheels/b0/4a/c8/9c580a0ea94dea641b8023456a1b178a7e4509d992224ed4aa
Successfully built emoji
You should consider upgrading via the '/Users/u14510182/Documents/python_for_nlp_stud/venv/bin/python -m pip install --upgrade pip' command.


In [334]:
import spacy_stanza

In [335]:
ru_stanza = spacy_stanza.load_pipeline("ru")

In [341]:
# txt = 'Я был в деревне целый день. И завтра тоже там буду.'
txt = 'Миновав галерею, я приблизился к арочному проему, за которым открывался зал с высоким сводом, и остановился.'
doc = ru_stanza(txt)

In [337]:
type(doc), doc

(spacy.tokens.doc.Doc, Я был в деревне целый день. И завтра тоже там буду.)

In [338]:
ru_stanza.pipe_names

[]

In [342]:
sents = list(doc.sents)
sents

[Миновав галерею, я приблизился к арочному проему, за которым открывался зал с высоким сводом, и остановился.]

In [343]:
displacy.render(sents[0], style='dep')

# CoNLL
Существует специальный текстовый формат для хранения разобранных предложений. Он называется CoNLL.
Разобранный текст выглядит в виде строк, где каждая строчка - разбор отдельного слова.
В рамках одной строки отображается вся информация по слову: лемма, морфологические признаки, порядковый номер слова, с каким словом оно связано и тип этой связи. Разделителем является `\t`.  
Предложения разделены `\n\n`.

Само название формата пошло от названия конференции (Conference on Computational Natural Language Learning), в рамках который проводились и проводятся соревнования по парсингу текста. 

In [245]:
!pip install spacy_conll

You should consider upgrading via the '/Users/u14510182/Documents/python_for_nlp_stud/venv/bin/python -m pip install --upgrade pip' command.


In [344]:
from spacy_conll import init_parser
import spacy_conll

In [345]:
ru_stanza_conll = init_parser(
    model_or_lang="ru", 
    parser="stanza", # Varians: 'spacy', 'stanza', 'udpipe'
    # parser_opts={"use_gpu": True, "verbose": False}, GPU turn off/on
)

In [346]:
txt = 'Я был в деревне целый день. И завтра тоже там буду.'
doc = ru_stanza_conll(txt)

In [348]:
# Получили разбор текста в формате CoNLL
conll_doc = doc._.conll_str 
print(conll_doc)

1	Я	я	PRON	PRON	Case=Nom|Number=Sing|Person=1|PronType=Prs	4	nsubj	_	_
2	был	быть	AUX	AUX	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	4	cop	_	_
3	в	в	ADP	ADP	_	4	case	_	_
4	деревне	деревня	NOUN	NOUN	Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing	0	root	_	_
5	целый	целый	ADJ	ADJ	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	6	amod	_	_
6	день	день	NOUN	NOUN	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	4	nmod	_	SpaceAfter=No
7	.	.	PUNCT	PUNCT	_	4	punct	_	_

1	И	и	CCONJ	CCONJ	_	4	cc	_	_
2	завтра	завтра	ADV	ADV	Degree=Pos	4	advmod	_	_
3	тоже	тоже	PART	PART	_	4	advmod	_	_
4	там	там	ADV	ADV	Degree=Pos	0	root	_	_
5	буду	быть	AUX	AUX	Aspect=Imp|Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin|Voice=Act	4	cop	_	SpaceAfter=No
6	.	.	PUNCT	PUNCT	_	4	punct	_	SpaceAfter=No



Что делать, если у нас есть распарсеный текст, сохраненный в формат CoNLL?  
В `spacy_conll` есть возможность из CoNLL получить объект SpaCy. После этого вам открываются все возможности работы с текстом от Spacy.

In [250]:
conll_parser = spacy_conll.ConllParser(ru_stanza_conll)

In [350]:
def prepare_conll(conll_text):
    clear_conll = []
    sents = conll_text.split('\n\n')
        
    for sent in sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        clear_conll.append('\n'.join(tree))
    return clear_conll

In [351]:
clear_conll_doc = prepare_conll(conll_doc)
type(clear_conll_doc), len(clear_conll_doc)

(list, 2)

In [352]:
clear_conll_doc[0]

'1\tЯ\tя\tPRON\tPRON\tCase=Nom|Number=Sing|Person=1|PronType=Prs\t4\tnsubj\t_\t_\n2\tбыл\tбыть\tAUX\tAUX\tAspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act\t4\tcop\t_\t_\n3\tв\tв\tADP\tADP\t_\t4\tcase\t_\t_\n4\tдеревне\tдеревня\tNOUN\tNOUN\tAnimacy=Inan|Case=Loc|Gender=Fem|Number=Sing\t0\troot\t_\t_\n5\tцелый\tцелый\tADJ\tADJ\tAnimacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing\t6\tamod\t_\t_\n6\tдень\tдень\tNOUN\tNOUN\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Sing\t4\tnmod\t_\tSpaceAfter=No\n7\t.\t.\tPUNCT\tPUNCT\t_\t4\tpunct\t_\t_'

In [353]:
sent = conll_parser.parse_conll_text_as_spacy(clear_conll_doc[0])
type(sent)

spacy.tokens.doc.Doc

In [356]:
sent[2]

в

In [357]:
doc_df = pd.DataFrame([get_features(token) for token in sent])
doc_df

,text,lemma,pos,tag,morph,dep,head,shape,is_alpha,is_stop
0,Я,я,PRON,PRON,"(Case=Nom, Number=Sing, Person=1, PronType=Prs)",nsubj,деревне,X,True,True
1,был,быть,AUX,AUX,"(Aspect=Imp, Gender=Masc, Mood=Ind, Number=Sin...",cop,деревне,xxx,True,True
2,в,в,ADP,ADP,(),case,деревне,x,True,True
3,деревне,деревня,NOUN,NOUN,"(Animacy=Inan, Case=Loc, Gender=Fem, Number=Sing)",ROOT,деревне,xxxx,True,False
4,целый,целый,ADJ,ADJ,"(Animacy=Inan, Case=Acc, Degree=Pos, Gender=Ma...",amod,день,xxxx,True,False
5,день,день,NOUN,NOUN,"(Animacy=Inan, Case=Acc, Gender=Masc, Number=S...",nmod,деревне,xxxx,True,False
6,.,.,PUNCT,PUNCT,(),punct,деревне,.,False,False


In [358]:
displacy.render(sent, style='dep')